In [ ]:
from pathlib import Path 
import os, dotenv, yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

dotenv.load_dotenv()
os.chdir(Path(config["pythonpath"]).expanduser())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from src.preprocessing.quality_process import compute_file_metrics

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer

VERSION = config["version"]
VARIATION = "nocomments"

DATA = f"data/good_syntax/verified_{VARIATION}_{VERSION}.jsonl"

MODEL_ID = config["model_id"]

codellama_tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    trust_remote_code=False,
)

if codellama_tokenizer.pad_token is None:
    codellama_tokenizer.pad_token = codellama_tokenizer.eos_token

# from quality_process import compute_file_metrics  # <-- uncomment and fix path

results = []
with open(DATA, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="Scoring rows"):
        row = json.loads(line)
        results.append(
            compute_file_metrics(
                row["filename"],
                row["content"],
                model_tokenizer=codellama_tokenizer,
            )
        )


df = pd.DataFrame(results)


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# StarCoder-like thresholds (tune if needed)
MAX_BYTES         = 200_000
MAX_NONASCII      = 0.20
ENC_MAX_RUN_CHARS = 1024
ENC_MAX_FRACTION  = 0.50
MAX_LINES_TOTAL   = 100_000
MAX_LINE_AVG_LEN  = 100
MAX_LINE_MAX_LEN  = 1_000
MIN_TOKENS_LANG   = 40      # language-token gate (Cryptol tokenizer)
MAX_TOKENS_LANG   = 10_000  # optional upper bound
MIN_TOKENS_MODEL  = 40      # only if you’ve populated num_tokens_model
MAX_TOKENS_MODEL  = 1600
MAX_HEXNUM_RATIO  = 0.20


# --- exact dedup (keep first occurrence of each sha1) ---
# mark duplicates (True means "is duplicate" => drop later)
dup_mask = df.duplicated(subset=["sha1"], keep="first")

# --- encoded data (StarCoder) ---
enc_mask = (df["enc_max_run"] > ENC_MAX_RUN_CHARS) | (df["enc_fraction"] > ENC_MAX_FRACTION)

# --- long-line filters (StarCoder) ---
longline_mask = (
    (df["lines"] > MAX_LINES_TOTAL) |
    (df["avg_line_len"] > MAX_LINE_AVG_LEN) |
    (df["max_line_len"] > MAX_LINE_MAX_LEN)
)

# --- binary-like content ---
binary_mask = df["binary_like"].fillna(False)

# --- non-ascii density ---
nonascii_mask = df["non_ascii_ratio"].fillna(0) > MAX_NONASCII

# --- size guardrail (bytes) ---
bytes_mask = df["bytes"].fillna(0) > MAX_BYTES

# --- language-token bounds ---
lang_small_mask = df["num_tokens_model"].fillna(0) < MIN_TOKENS_MODEL
lang_large_mask = df["num_tokens_model"].fillna(0) > MAX_TOKENS_MODEL

# --- shingles exist (needed for Jaccard) ---
no_shingles_mask = df["num_shingles"].fillna(0) <= 0

# --- numeric/hex blob concentration ---
hexnum_mask = df["hexnum_ratio"].fillna(0) > MAX_HEXNUM_RATIO

# --- model-token gate (only apply where available) ---
if "num_tokens_model" in df.columns:
    model_small_mask = df["num_tokens_model"].fillna(np.inf) < MIN_TOKENS_MODEL
else:
    model_small_mask = pd.Series(False, index=df.index)


In [ ]:
# Combine all hard-drop reasons
drop_mask = (
    dup_mask |
    enc_mask |
    longline_mask |
    binary_mask |
    nonascii_mask |
    bytes_mask |
    lang_small_mask |
    lang_large_mask |
    no_shingles_mask |
    hexnum_mask |
    model_small_mask
)

# Optional: compute a human-readable fail reason (first rule that tripped)
def first_reason(i):
    if dup_mask.iat[i]:          return "exact_duplicate"
    if enc_mask.iat[i]:          return "encoded_data"
    if longline_mask.iat[i]:     return "long_lines"
    if binary_mask.iat[i]:       return "binary_like"
    if nonascii_mask.iat[i]:     return "too_much_nonascii"
    if bytes_mask.iat[i]:        return "too_large_bytes"
    if lang_small_mask.iat[i]:   return "too_few_lang_tokens"
    if lang_large_mask.iat[i]:   return "too_many_lang_tokens"
    if no_shingles_mask.iat[i]:  return "no_shingles"
    if hexnum_mask.iat[i]:       return "hexnum_blob"
    if model_small_mask.iat[i]:  return "too_few_model_tokens"
    return "ok"

df = df.copy()
df["quality_ok"] = ~drop_mask
df["fail_reason"] = [first_reason(i) for i in range(len(df))]


In [ ]:
dedup_cols = [
    "filename", "sha1",
    # size/lines
    "bytes", "lines", "avg_line_len", "max_line_len",
    # content/encoding
    "non_ascii_ratio", "binary_like",
    "enc_total_matched", "enc_max_run", "enc_fraction",
    "enc_hits_base64", "enc_hits_hexbytes", "enc_hits_unicode",
    # tokens/shingles
    "num_tokens_lang", "k_shingle", "num_shingles", "hexnum_ratio",
    # model tokens (optional)
    "num_tokens_model",
    # path heuristic & status
    "quality_ok", "fail_reason",
]

candidate_df = df.loc[df["quality_ok"], dedup_cols].reset_index(drop=True)
similar_process_df = pd.read_json(DATA, lines=True)
similar_process_df = similar_process_df[similar_process_df['filename'].isin(candidate_df['filename'])].reset_index(drop=True)

put_back_path = Path("data/dropped/files_to_put_back.csv")
'''
if put_back_path.exists():
    put_back_set = pd.read_csv(put_back_path)
    put_back_filenames = set(put_back_set["filename"].dropna().tolist())
else:
    put_back_filenames = set()

for fname in put_back_filenames:
    if fname in df['filename'].values:
        candidate_df = pd.concat([candidate_df, df[df['filename'] == fname][dedup_cols]], ignore_index=True)
        candidate_df.loc[candidate_df['filename'] == fname, 'quality_ok'] = True
        '''
similar_process_df.head()

In [ ]:
print("[summary] total:", len(df))
print("[summary] kept :", int(df["quality_ok"].sum()))
print("[summary] dropped:", int((~df["quality_ok"]).sum()))
print("[summary] drop reasons:")
print(df.loc[~df["quality_ok"], "fail_reason"].value_counts())


In [ ]:
dropped = df[df["quality_ok"] == False].copy().reset_index(drop=True)
df.info()


In [ ]:
#review_data_set = df[~df['fail_reason'].isin(['ok', 'exact_duplicate'])].copy().reset_index(drop=True)
#out_path = Path(f"data/dropped/review_files_{VARIATION}_{VERSION}.csv")
#out_path.parent.mkdir(parents=True, exist_ok=True)  # create dirs if missing
#review_data_set.to_csv(out_path, index=False)


In [ ]:
from src.preprocessing.similiar_process import run_from_dataframe

# candidate_df must have an absolute-path 'filename' column.
df_files, df_pairs, similar_files = run_from_dataframe(
    similar_process_df,
    filename_col="filename",
    content_col="content",
    out_dir="minhash_outputs",
)


In [ ]:
df_files.head()

In [ ]:
from src.preprocessing.cluster_process import run_clustering

# If you already have df_files/df_pairs in memory:
df_keep, df_drop, df_clusters = run_clustering(
    df_files=df_files,          # from similiar_process
    df_pairs=df_pairs,          # from similiar_process
    jaccard_keep_threshold=0.70,
    out_dir="minhash_outputs",
    content_lookup=None,        # or {filename: raw_text} if you want text-derived penalties
    save_outputs=True
)

In [ ]:
dataset = df[df['filename'].isin(df_keep['filename'].tolist())].copy()
df.loc[
    df['filename'].isin(df_drop['filename'].tolist()),
    'quality_ok'
    ] = False
df.loc[
    df['filename'].isin(df_drop['filename'].tolist()),
    'fail_reason'
    ] = 'similiar_file_exists'

dropped = df[~df['filename'].isin(dataset['filename'].tolist())].copy().reset_index(drop=True)


In [ ]:
#dropped.to_csv(f"data/dropped/{VARIATION}_dropped_files_{VERSION}.csv", index=False)

In [ ]:
import re

verified_files = set(dataset["filename"].unique())

#all_files_df = pd.read_json(DATA1, lines=True)
all_files_filtered_df = similar_process_df[similar_process_df["filename"].isin(verified_files)].reset_index(drop=True)

pattern = r"/\d{3}_testCase"   # slash + exactly 3 digits + "_testCase"
mask = all_files_filtered_df["filename"].astype(str).str.contains(pattern, regex=True, na=False)

test_case_df = all_files_filtered_df[mask].copy().reset_index(drop=True)
all_files_filtered_df = all_files_filtered_df[~mask].reset_index(drop=True)

mask = all_files_filtered_df["filename"].astype(str).str.contains(r"/tests/", regex=True, na=False)
all_files_filtered_df = all_files_filtered_df[~mask]

out_path = Path(f"data/clean_datasets/{VARIATION}_{VERSION}.jsonl")
out_path.parent.mkdir(parents=True, exist_ok=True)  # create dirs if missing
all_files_filtered_df.to_json(out_path, lines=True, orient="records")
test_case_df.to_json(f"data/clean_datasets/testCases_{VARIATION}_{VERSION}.jsonl", lines=True, orient="records")

In [ ]:
df.to_csv(f"data/{VARIATION}_file_metrics_{VERSION}.csv", index=False)